In [1]:
import open3d as o3d
import numpy as np
import sys
import open3d.visualization.gui as gui


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.insert(0,'sample')

In [3]:
from read_datas import ReadDatas

In [4]:
datas =  ReadDatas('Point Cloud Data\Corner.ply')

In [5]:
o3d.visualization.draw_geometries([datas.datas])


In [6]:
datas.gpf_ground_extraction()

In [7]:
ground = datas.np_to_o3d(datas.ground)


In [9]:
point = datas.seed_select(ground)

[Open3D INFO] Picked point #15100 (-14., -13., -1.6) to add in queue.
[Open3D INFO] Picked point #10521 (-8.4, -13., -1.6) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #20020 (-4.2, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #23001 (-2.6, -12., -1.6) to add in queue.
[Open3D INFO] Remove picked point #23001 from pick queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #22127 (-2.8, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #39976 (0.52, -11., -1.5) to add in queue.
[Open3D INFO] Picked point #60074 (4.3, -8.0, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #50962 (5.3, -6.0, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求

In [10]:
point

[15100, 10521, 20020, 22127, 39976, 60074, 50962, 69858]

In [31]:
geometrys = datas.find_no_paves(ground)

In [12]:
def calculate_linear_density(ground):
    nndist = ground.compute_nearest_neighbor_distance()
    nndist = np.array(nndist)
    density = np.mean(nndist)
    return density


In [14]:
def driving_path_generation(point,ground):
    density = calculate_linear_density(ground)
    driving_point = ground.select_by_index(point)
    driving_point = np.array(driving_point.points)

    ground = ground.select_by_index(point, invert=True)
    driving_distance = np.array([np.linalg.norm(driving_point[p] - driving_point[p+1]) for p in range(len(driving_point) - 1 )])
    driving_track = np.array([[0,0,0]])

    for i,d in enumerate(driving_distance):
        num = int(d / density)

        driving_track_x = np.linspace(driving_point[i,0],driving_point[i+1,0],num)
        driving_track_y = np.linspace(driving_point[i,1],driving_point[i+1,1],num)
        driving_track_z = np.linspace(driving_point[i,2],driving_point[i+1,2],num)
        new_points = np.array([driving_track_x,driving_track_y,driving_track_z]).T

        driving_track = np.concatenate((driving_track,new_points),axis=0)
    driving_track = np.delete(driving_track,0,axis=0)
    return driving_track


In [15]:
driving_track = driving_path_generation(point,ground)

In [16]:
driving_track_o3d = datas.np_to_o3d(driving_track)


In [17]:
ground += driving_track_o3d

In [20]:
def find_nearest_point(clouds1,clouds2,neighbour_number = 1):

    neighbour_number = 1
    kdtree = o3d.geometry.KDTreeFlann(clouds1)
    number = len(ground.points)

    point_neighbours = np.zeros((number, neighbour_number))
    for ik in range(number):
        [_, point[0], _] = kdtree.search_knn_vector_3d(clouds2.points[ik], neighbour_number)  # K近邻搜索
        point_neighbours[ik, :] = point[0]
    return point_neighbours


In [21]:
point_neighbours = find_nearest_point(driving_track_o3d,ground)


In [2]:
def driving_path_extraction(ground,driving_track):
    ground_np = np.asarray(ground.points)
    ground_nom = ground.estimate_normals()
    ground_nom = np.asarray(ground.normals)
    theta_threshold = 30
    cosine_threshold = np.cos(np.deg2rad(theta_threshold))
    paves = np.array([[0,0,0]])
    for i,p in enumerate(driving_track):
        slim = ground_np[point_neighbours[:,0] == i]
        if len(slim) > 0:
            slim_distance = np.sqrt(np.sum(np.power(slim - driving_track[i,:],2),axis = 1))
            p = slim_distance.argmin()
            slim_nom = slim - driving_track[i,:]
            slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] = slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] * -1
            slim = slim[slim_distance.argsort()]
            slim_nor = ground_nom[point_neighbours[:,0] == i]
            p = slim_nor[p,:]
            slim_nor = slim_nor[slim_distance.argsort()]
            slim_distance.sort()
            a_jiao = np.fabs(np.dot(slim_nor, p))
            b = np.diff(slim[:,2])
            b =np.append(b,1)
            a = a_jiao > cosine_threshold
            d = slim[a]
            paves = np.concatenate((paves,d),axis=0)
    pave = np.delete(pave,0)        

In [32]:
seed = datas.find_index(paves,geometrys)

In [29]:
seed = []

In [33]:
for i in geometrys:
    a = np.argwhere((paves == i).all(axis=1)).ravel()
    if len(a)>0:
        seed.append(a[0])


In [34]:
paves = np.delete(paves,seed,0)


In [35]:
paves = datas.np_to_o3d(paves)

In [36]:
o3d.visualization.draw_geometries([paves])


In [38]:
def euclidean_cluster(cloud,point, tolerance=0.2, min_cluster_size=100, max_cluster_size=1000000000000):
    """
    欧式聚类
    :param cloud:输入点云
    :param tolerance: 设置近邻搜索的搜索半径（也即两个不同聚类团点之间的最小欧氏距离）
    :param min_cluster_size:设置一个聚类需要的最少的点数目
    :param max_cluster_size:设置一个聚类需要的最大点数目
    :return:聚类个数
    """

    kdtree = o3d.geometry.KDTreeFlann(cloud)  # 对点云建立kd树索引

    num_points = len(cloud.points)
    processed = [-1] * num_points  # 定义所需变量
    clusters = []  # 初始化聚类
    # 遍历各点
    while point:
        if processed[point[0]] == 1:  # 如果该点已经处理则跳过
            continue
        seed_queue = []  # 定义一个种子队列
        sq_idx = 0
        seed_queue.append(point[0])  # 加入一个种子点
        processed[point[0]] = 1

        while sq_idx < len(seed_queue):

            k, nn_indices, _ = kdtree.search_radius_vector_3d(cloud.points[seed_queue[sq_idx]], tolerance)

            if k == 1:  # k=1表示该种子点没有近邻点
                sq_idx += 1
                continue
            for j in range(k):

                if nn_indices[j] == num_points or processed[nn_indices[j]] == 1:
                    continue  # 种子点的近邻点中如果已经处理就跳出此次循环继续
                seed_queue.append(nn_indices[j])
                processed[nn_indices[j]] = 1

            sq_idx += 1

        if max_cluster_size > len(seed_queue) > min_cluster_size:
            clusters.append(seed_queue)
            point = False

    return seed_queue

In [152]:
if __name__ == '__main__':
    # --------------------------加载点云数据------------------------------
    pcd = paves
    # ---------------------------欧式聚类--------------------------------
    ec = euclidean_cluster(pcd, tolerance=0.1, min_cluster_size=1000, max_cluster_size=100000000)
    # -------------------------聚类结果分类保存---------------------------

    for i in range(len(ec)):
        ind = ec[i]
        clusters_cloud = pcd.select_by_index(ind)
        file_name = "euclidean_cluster" + str(i + 1) + ".pcd"
        o3d.io.write_point_cloud(file_name, clusters_cloud)

In [39]:
pcd = paves
    # ---------------------------欧式聚类--------------------------------
ec = euclidean_cluster(pcd,point, tolerance=0.1, min_cluster_size=1000, max_cluster_size=100000000)

In [42]:
ec = pcd.select_by_index(ec)


In [3]:
o3d.visualization.draw_geometries([ec])


NameError: name 'o3d' is not defined